<a href="https://colab.research.google.com/github/path-0f-misantrope/most_uselles_musor_final/blob/main/result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install scikit-learn
!pip install nlpaug
!pip install transformers


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset


In [44]:
df = pd.read_csv('/content/разметка_готовая.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1970 entries, 0 to 1969
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   comment                               1970 non-null   object 
 1   Нравится скорость отработки заявок    1970 non-null   int64  
 2   Нравится качество выполнения заявки   1969 non-null   float64
 3   Нравится качество работы сотрудников  1970 non-null   int64  
 4   Понравилось выполнение заявки         1969 non-null   float64
 5   Вопрос решен                          1967 non-null   float64
dtypes: float64(3), int64(2), object(1)
memory usage: 92.5+ KB


In [4]:

text_col = 'comment'
label_cols = [
    'Нравится скорость отработки заявок',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Вопрос решен',
    "Понравилось выполнение заявки"
]
df = df[[text_col] + label_cols].dropna()
df = df[df[label_cols].sum(axis=1) > 0]

In [28]:
import nlpaug.augmenter.word as naw

aug = naw.ContextualWordEmbsAug(
    model_path='DeepPavlov/rubert-base-cased',
    action="substitute"
)


def augment_rare_classes(df, label_cols, target_count=300, augmenter=None, num_aug=6):
    dfs = [df.copy()]

    for label in label_cols:
        df_label = df[df[label] == 1]
        n_current = len(df_label)
        n_to_add = target_count - n_current
        if n_to_add <= 0:
            continue

        texts_to_augment = df_label['comment'].tolist()
        labels_to_copy = df_label[label_cols]

        augmented_texts = []
        augmented_labels = []

        # Генерируем аугментации в цикле, пока не достигнем n_to_add
        while len(augmented_texts) < n_to_add:
            for idx, text in enumerate(texts_to_augment):
                new_text = augmenter.augment(text)
                augmented_texts.append(new_text)
                augmented_labels.append(labels_to_copy.iloc[idx].values)
                if len(augmented_texts) >= n_to_add:
                    break

        augmented_df = pd.DataFrame({'text': augmented_texts})
        for i, col in enumerate(label_cols):
            augmented_df[col] = [label[i] for label in augmented_labels]

        dfs.append(augmented_df)

    return pd.concat(dfs).sample(frac=1).reset_index(drop=True)
df = augment_rare_classes(df, label_cols, target_count=600, augmenter=aug, num_aug=6)


In [29]:
df[label_cols].sum()

,0
Нравится скорость отработки заявок,1355.0
Нравится качество выполнения заявки,702.0
Нравится качество работы сотрудников,747.0
Вопрос решен,1909.0
Понравилось выполнение заявки,782.0




```


Нравится скорость отработки заявок	1355.0
Нравится качество выполнения заявки	702.0
Нравится качество работы сотрудников	747.0
Вопрос решен	1909.0
Понравилось выполнение заявки	782.0
формат
```



In [32]:
df.head()

,comment,Нравится скорость отработки заявок,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Вопрос решен,Понравилось выполнение заявки,text
0,"очень благодарна, компетентному сотруднику и г...",0.0,0.0,1.0,0.0,0.0,NaN
1,NaN,0.0,0.0,0.0,1.0,1.0,[ну здесь и целом был кто что заявку [UNK] зде...
2,"отлично, быстро, надёжно!",1.0,0.0,0.0,0.0,0.0,NaN
3,NaN,1.0,1.0,1.0,1.0,1.0,[ремонтные по по государственной заявке ведутс...
4,"спасибо, решили мой вопрос на месте в офисе",0.0,0.0,0.0,1.0,0.0,NaN


In [33]:

df['comment'] = df['comment'].fillna(df['text'])
X = df['comment'].values
y = df[label_cols].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [34]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

class CommentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(self.texts[idx],
                             truncation=True,
                             padding='max_length',
                             max_length=128,
                             return_tensors="pt")
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

train_dataset = CommentDataset(X_train, y_train)
test_dataset = CommentDataset(X_test, y_test)


In [36]:

import numpy as np


label_counts = np.sum(y_train, axis=0)
total_counts = y_train.shape[0]

pos_weights = total_counts / (label_counts + 1e-6)
pos_weights = torch.tensor(pos_weights, dtype=torch.float32)


In [37]:
from transformers import Trainer
import torch.nn as nn
class WeightedTrainer(Trainer):
    def __init__(self, pos_weight, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight.to(self.args.device)  # переводим веса на нужное устройство
        self.loss_fn = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")

        # Переводим labels на то же устройство, что и модель
        labels = labels.to(model.device)

        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fn(logits, labels.float())

        return (loss, outputs) if return_outputs else loss



In [41]:
import os
os.environ["WANDB_DISABLED"] = "true"


model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=100,
    #evaluation_strategy="epoch"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    pos_weight=pos_weights,

)


trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-37-28f5e88fa894>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Step,Training Loss
100,0.872100
200,0.733600
300,0.700000
400,0.565100
500,0.525600
600,0.486400
700,0.411200
800,0.394700
900,0.312800
1000,0.255900


TrainOutput(global_step=1698, training_loss=0.37628488799006415, metrics={'train_runtime': 444.5089, 'train_samples_per_second': 30.492, 'train_steps_per_second': 3.82, 'total_flos': 891575825739264.0, 'train_loss': 0.37628488799006415, 'epoch': 6.0})

In [42]:
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
preds= trainer.predict(test_dataset)

y_pred = (preds.predictions > 0.5).astype(int)
y_true = np.asarray(y_test).astype(int)

# Заменим 2 → 1
y_true[y_true == 2] = 1

print("Unique values in y_true (after fix):", np.unique(y_true))
print(classification_report(y_true, y_pred, target_names=label_cols))






Unique values in y_true (after fix): [0 1]
                                      precision    recall  f1-score   support

  Нравится скорость отработки заявок       0.93      0.92      0.93       268
 Нравится качество выполнения заявки       0.79      0.78      0.78       138
Нравится качество работы сотрудников       0.77      0.75      0.76       150
                        Вопрос решен       0.94      0.91      0.93       383
       Понравилось выполнение заявки       0.72      0.72      0.72       160

                           micro avg       0.86      0.85      0.86      1099
                           macro avg       0.83      0.82      0.82      1099
                        weighted avg       0.86      0.85      0.86      1099
                         samples avg       0.88      0.87      0.86      1099




```
                                      precision    recall  f1-score   support

  Нравится скорость отработки заявок       0.93      0.92      0.93       268
 Нравится качество выполнения заявки       0.79      0.78      0.78       138
Нравится качество работы сотрудников       0.77      0.75      0.76       150
                        Вопрос решен       0.94      0.91      0.93       383
       Понравилось выполнение заявки       0.72      0.72      0.72       160

                           micro avg       0.86      0.85      0.86      1099
                           macro avg       0.83      0.82      0.82      1099
                        weighted avg       0.86      0.85      0.86      1099
                         samples avg       0.88      0.87      0.86      1099
                         

```



In [43]:
from sklearn.metrics import roc_auc_score

# preds.predictions — это логиты, подаём их как вероятности
# y_true — бинарные истинные метки (после исправлений)
# y_pred_proba — вероятности принадлежности к классу (до округления)

y_pred_proba = preds.predictions
y_true = np.asarray(y_test).astype(int)
y_true[y_true == 2] = 1  # На всякий случай ещё раз

# ROC-AUC по каждому классу
roc_auc_per_class = roc_auc_score(y_true, y_pred_proba, average=None)

# Средние оценки
roc_auc_macro = roc_auc_score(y_true, y_pred_proba, average="macro")
roc_auc_micro = roc_auc_score(y_true, y_pred_proba, average="micro")
roc_auc_weighted = roc_auc_score(y_true, y_pred_proba, average="weighted")

# Выводим
print("ROC-AUC по каждому классу:")
for label, score in zip(label_cols, roc_auc_per_class):
    print(f"{label}: {score:.3f}")

print("\nROC-AUC (macro):", round(roc_auc_macro, 3))
print("ROC-AUC (micro):", round(roc_auc_micro, 3))
print("ROC-AUC (weighted):", round(roc_auc_weighted, 3))


ROC-AUC по каждому классу:
Нравится скорость отработки заявок: 0.974
Нравится качество выполнения заявки: 0.941
Нравится качество работы сотрудников: 0.905
Вопрос решен: 0.964
Понравилось выполнение заявки: 0.889

ROC-AUC (macro): 0.934
ROC-AUC (micro): 0.945
ROC-AUC (weighted): 0.944




```
ROC-AUC по каждому классу:
Нравится скорость отработки заявок: 0.974
Нравится качество выполнения заявки: 0.941
Нравится качество работы сотрудников: 0.905
Вопрос решен: 0.964
Понравилось выполнение заявки: 0.889

ROC-AUC (macro): 0.934
ROC-AUC (micro): 0.945
ROC-AUC (weighted): 0.944

```

